In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### **Generate Random weather data**

In [ ]:
# temperature
# precipitation (percentage)
# rain or not

In [43]:
data = pd.read_csv ("weather_data.csv")
data

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
...,...,...,...,...,...,...
1456,2015-12-27,8.6,4.4,1.7,2.9,rain
1457,2015-12-28,1.5,5.0,1.7,1.3,rain
1458,2015-12-29,0.0,7.2,0.6,2.6,fog
1459,2015-12-30,0.0,5.6,-1.0,3.4,sun


In [44]:
train_set_size = 1200

In [45]:
temperatures = (data.temp_max.head (train_set_size).values + data.temp_min.head (train_set_size).values)/2
temperatures

array([8.9 , 6.7 , 9.45, ..., 9.45, 7.8 , 7.25])

In [46]:
precipitation = data.precipitation.head (train_set_size).values
precipitation

array([ 0. , 10.9,  0.8, ...,  0. , 14. ,  3.3])

In [49]:
rainy = data.weather.head (train_set_size).values
rainy

array(['drizzle', 'rain', 'rain', ..., 'sun', 'rain', 'rain'],
      dtype=object)

In [55]:
temp_rainy = []
ppt_rainy = []

temp_not_rainy = []
ppt_not_rainy = []

for i in range (len (rainy)) :
    if rainy [i] == 'rain' :
        temp_rainy.append (temperatures [i])
        ppt_rainy.append (precipitation [i])
    
    else :
        temp_not_rainy.append (temperatures [i])
        ppt_not_rainy.append (precipitation [i])

In [59]:
stats_temp_rainy = pd.Series (temp_rainy).describe ()

mean_temp_rainy = stats_temp_rainy ['mean']
std_temp_rainy = stats_temp_rainy ['std']
print ("Rainy :")
print (stats_temp_rainy, mean_temp_rainy, std_temp_rainy)

stats_temp_not_rainy = pd.Series (temp_not_rainy).describe ()

mean_temp_not_rainy = stats_temp_not_rainy ['mean']
std_temp_not_rainy = stats_temp_not_rainy ['std']

print ("\nNot Rainy :")
print (stats_temp_not_rainy, mean_temp_not_rainy, std_temp_not_rainy)

Rainy :
count    551.000000
mean      10.431670
std        4.211839
min        1.950000
25%        7.200000
50%        9.750000
75%       12.800000
max       26.700000
dtype: float64 10.431669691470056 4.21183934423038

Not Rainy :
count    649.000000
mean      12.974422
std        6.744912
min       -3.800000
25%        7.750000
50%       14.400000
75%       18.600000
max       26.350000
dtype: float64 12.97442218798151 6.744912395481265


In [60]:
stats_ppt_rainy = pd.Series (ppt_rainy).describe ()

mean_ppt_rainy = stats_ppt_rainy ['mean']
std_ppt_rainy = stats_ppt_rainy ['std']
print ("Rainy :")
print (stats_ppt_rainy, mean_ppt_rainy, std_ppt_rainy)

stats_ppt_not_rainy = pd.Series (ppt_not_rainy).describe ()

mean_ppt_not_rainy = stats_ppt_not_rainy ['mean']
std_ppt_not_rainy = stats_ppt_not_rainy ['std']

print ("\nNot Rainy :")
print (stats_ppt_not_rainy, mean_ppt_not_rainy, std_ppt_not_rainy)

Rainy :
count    551.000000
mean       6.245735
std        8.146650
min        0.000000
25%        0.800000
50%        3.300000
75%        8.500000
max       55.900000
dtype: float64 6.245735027223231 8.146649894539916

Not Rainy :
count    649.000000
mean       0.342681
std        2.172609
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       23.900000
dtype: float64 0.34268104776579356 2.1726085671550326


In [50]:
rainy_days = 0
total_days = len (rainy)
for i in range (len (rainy)) :
    if rainy [i] == 'rain' :
        rainy_days = rainy_days + 1

prior_prob_rainy = rainy_days/total_days
prior_prob_not_rainy = 1 - prior_prob_rainy

In [51]:
def gaussian_likelihood (x, mean, std) :
    gaussian = 1/math.sqrt (2 * math.pi * std**2)
    gaussian = gaussian + math.exp (-(x - mean)**2/(2 * std**2))
    
    return gaussian

In [52]:
test_size = len (data) - train_set_size
test_temp = (data.temp_max.tail (test_size).values + data.temp_min.tail (test_size).values)/2
test_ppt = data.precipitation.tail (test_size).values

test_rainy = data.weather.tail (test_size).values

In [62]:
import math
pred_rainy = []

for i in range (len (test_temp)) :
    likelihood_rainy = prior_prob_rainy * gaussian_likelihood (test_temp [i], mean_temp_rainy, std_temp_rainy) * gaussian_likelihood (test_ppt [i], mean_ppt_rainy, std_ppt_rainy)
    likelihood_not_rainy = prior_prob_not_rainy * gaussian_likelihood (test_temp [i], mean_temp_not_rainy, std_temp_not_rainy) * gaussian_likelihood (test_ppt [i], mean_ppt_not_rainy, std_ppt_not_rainy)
    
    if likelihood_rainy > likelihood_not_rainy :
        pred_rainy.append ("rain")
        
    else :
        pred_rainy.append ("no rain")

In [67]:
rainy = 0
not_rainy = 0
for i in range (len (pred_rainy)) :
    if pred_rainy [i] == test_rainy [i] and pred_rainy [i] == "rain" :
        rainy = rainy + 1
    elif pred_rainy [i] != test_rainy [i] and pred_rainy [i] != "rain" and test_rainy [i] != 'rain' :
        not_rainy = not_rainy + 1
        
print ((rainy + not_rainy)/(len (pred_rainy)))

0.8045977011494253
